In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter
from sklearn.model_selection import train_test_split

from konlpy.tag import Komoran

In [24]:
data = pd.read_csv('../data/total_data.csv', encoding='UTF8')

In [25]:
display(data)

,title,description,url,data_type,source,ori_label,ori_source,label
0,충청남도 당진시_산업단지 입주기업 현황_20220304,"충청남도 당진시 산업단지 입주 기업 현황입니다.(연번, 단지명, 회사명, 업종명, ...",https://www.data.go.kr/data/15106898/fileData.do,텍스트,충청남도 당진시,산업·통상·중소기업,공공데이터포털,제조소비
1,전라남도 강진군_미곡처리장현황_20221006,"전라남도 강진군 미곡처리장 현황에 대한 데이터로소 시군, 명칭, 소재지, 관리부서,...",https://www.data.go.kr/data/15106993/fileData.do,텍스트,전라남도 강진군,농림,공공데이터포털,식품건강
2,경상남도 거창군_월별 코로나 확진자 현황_20221007,경상남도 거창군 월별 코로나19 관련 확진자 및 사망자 수 현황 데이터로 월별 확진...,https://www.data.go.kr/data/15098865/fileData.do,텍스트,경상남도 거창군,보건,공공데이터포털,보건의료
3,충청남도 부여군_1인당지방세부담액_20221006,상기 데이터는 연도별 주민 1인당 또는 세대 당 부담된 지방세액에 대한 정보로 조...,https://www.data.go.kr/data/15080007/fileData.do,텍스트,충청남도 부여군,재정·세제·금융,공공데이터포털,경제금융
4,전라남도 보성군_담배소매인지정현황_20221006,"전라남도 보성군 담배소매인지정현황에 관한 공공데이터로 업소명, 지번주소, 도로명주소...",https://www.data.go.kr/data/15035564/fileData.do,텍스트,전라남도 보성군,일반공공행정,공공데이터포털,행정법률
...,...,...,...,...,...,...,...,...
127046,서울시 구로구 식품첨가물제조업 인허가 정보,구로구의 식품을 제조 또는 가공할 때 기호가치를 향상시키거나 영양가치를 높일목적의 ...,https://data.seoul.go.kr/dataList/OA-18293/S/1...,"SHEET,OpenAPI",구로구 스마트도시정책관 빅데이터담당관,보건,서울데이터광장,보건의료
127047,서울시 송파구 국제회의시설업 인허가 정보,송파구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16990/S/1...,"SHEET,OpenAPI",송파구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
127048,서울시 중구 국제회의시설업 인허가 정보,중구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설을...,https://data.seoul.go.kr/dataList/OA-16968/S/1...,"SHEET,OpenAPI",중구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광
127049,서울시 강남구 국제회의시설업 인허가 정보,강남구의 컨벤션센터 등 대규모 관광 수요를 유발하는 국제회의를 개최할 수 있는 시설...,https://data.seoul.go.kr/dataList/OA-16989/S/1...,"SHEET,OpenAPI",강남구 스마트도시정책관 빅데이터담당관,문화/관광,서울데이터광장,문화관광


In [23]:
data['description'].isnull().sum()

0

In [5]:
tokenizer = Komoran()

In [18]:
def preprocess(df):
    data['description'].dropna(axis=0)
    print("replace start..")
    for i in range(len(data)):
        data['description'][i] = str(data['description'][i]).replace('\n','').replace('\t','').replace('\r','').replace('.', '').replace(',','').replace("'","").replace('·', ' ').replace('=','').replace('·','')
    print("done..")
    
    print("tokenizing start..")
    description_tokenized = [[token+"/"+POS for token, POS in tokenizer.pos(des_)] for des_ in data['description']]
    exclusion_tags = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC',
                      'SF', 'SP', 'SS', 'SE', 'SO', 'EF', 'EP', 'EC', 'ETN', 'ETM',
                      'XSN', 'XSV', 'XSA']
    
    f = lambda x: x in exclusion_tags
    
    des_preprocessed = []
    for i in range(len(description_tokenized)):
        temp = []
        for j in range(len(description_tokenized[i])):
            if f(description_tokenized[i][j].split('/')[1]) is False:
                temp.append(description_tokenized[i][j].split('/')[0])
        des_preprocessed.append(temp)
    print("done..")
    
    words = np.concatenate(des_preprocessed).tolist()
    counter = Counter(words)
    counter = counter.most_common(20000-1)
    vocab = ['UNK'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['UNK'] for word in wordlist]
    
    return des_preprocessed, word_to_index

In [ ]:
for i in range(len(data)):
    try:
        Komoran.pos(data['description'][i])
    except:
        print(f'{i} 번째 error발생')

In [19]:
des_result, word_to_index = preprocess(data)

replace start..
done..
tokenizing start..
done..


In [21]:
len(des_result)

127051

In [26]:
len(word_to_index)

20000

In [27]:
def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['UNK'] for word in wordlist]

In [32]:
des_index = []
for i in range(len(des_result)):
    des_index.append(wordlist_to_indexlist(des_result[i]))

In [36]:
len(des_index)

127051

In [37]:
import pickle

In [38]:
with open("../data/des_embedding_bow.pkl","wb") as f:
    pickle.dump(des_index, f)

In [39]:
with open("../data/des_embedding_bow.pkl","rb") as f:
    emb_load = pickle.load(f)

In [43]:
len(emb_load)

127051